In [204]:
import pandas as pd
import json
from google.cloud import storage
from google.cloud import bigquery
client = bigquery.Client()

# Load data

## Excel extracts

In [205]:
pcp_sample_cas_ms = pd.read_excel("Data_msr_v2/samples-RMPCP-CAS.xlsx")
pcp_sample_cosmetochem_ms = pd.read_excel("Data_msr_v2/samples-RMPCP-COSMETOCHEM.xlsx")
pcp_sample_echa_ms = pd.read_excel("Data_msr_v2/samples-RMPCP-ECHA.xlsx")
pcp_sample_lims_ms = pd.read_excel("Data_msr_v2/samples-RMPCP-LIMS-01AUG.xlsx")
pcp_test_lims_ms = pd.read_excel("Data_msr_v2/Tests-RMPCP-LIMS-01AUG.xlsx")

In [206]:
pcp_sample_cas_ms["id"] = pcp_sample_cas_ms["id"].astype(str)
pcp_sample_cas_ms["external_id"] = pcp_sample_cas_ms["external_id"].astype(str)
pcp_sample_echa_ms["id"] = pcp_sample_echa_ms["id"].astype(str)
pcp_sample_echa_ms["external_id"] = pcp_sample_echa_ms["external_id"].astype(str)
pcp_sample_lims_ms["id"] = pcp_sample_lims_ms["id"].astype(str)
pcp_sample_lims_ms["external_id"] = pcp_sample_lims_ms["external_id"].astype(str)
pcp_sample_cosmetochem_ms["id"] = pcp_sample_cosmetochem_ms["id"].astype(str)
pcp_sample_cosmetochem_ms["external_id"] = pcp_sample_cosmetochem_ms["external_id"].astype(str)
pcp_test_lims_ms["id"] = pcp_test_lims_ms["id"].astype(str)
pcp_test_lims_ms["external_id"] = pcp_test_lims_ms["external_id"].astype(str)

## SQL

In [207]:
with open(f"SQL/msr_v2/get_sample_cas.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_sample_cas_dwh = query_job.to_dataframe().drop_duplicates()

In [208]:
with open(f"SQL/msr_v2/get_sample_cas_sc_null_allowed.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_sample_cas_dwh_sc_null_allowed = query_job.to_dataframe().drop_duplicates()

In [209]:
with open(f"SQL/msr_v2/get_sample_cas_sc_null_allowed_aa_cas_allowed.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_sample_cas_dwh_sc_null_allowed_aa_cas_allowed = query_job.to_dataframe().drop_duplicates()

In [210]:
with open(f"SQL/msr_v2/get_sample_echa.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_sample_echa_dwh = query_job.to_dataframe().drop_duplicates()

In [211]:
with open(f"SQL/msr_v2/get_sample_aa.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_sample_lims_dwh = query_job.to_dataframe().drop_duplicates()

In [212]:
with open(f"SQL/msr_v2/get_sample_cosmetochem.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_sample_cosmetochem_dwh = query_job.to_dataframe().drop_duplicates()

In [213]:
with open(f"SQL/msr_v2/get_sample_aa_no_cas.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_sample_lims_dwh_no_cas = query_job.to_dataframe().drop_duplicates()

In [214]:
with open(f"SQL/msr_v2/get_sample_aa_no_cas_sc_null_allowed.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_sample_lims_dwh_no_cas_sc_null_allowed = query_job.to_dataframe().drop_duplicates()

In [215]:
with open(f"SQL/msr_v2/get_test_aa.sql", "r") as sql_file:
    sql_template = sql_file.read()

query_job = client.query(sql_template)
pcp_test_lims_dwh = query_job.to_dataframe().drop_duplicates()

## GA function

In [216]:
def gap_analysis_dataframes(df1: pd.DataFrame, df2: pd.DataFrame):
    """
    Performs a comprehensive gap analysis between two pandas DataFrames.

    This function provides useful information to assess the data gaps, including:
    - Number of lines in both DataFrames.
    - Percentage of NaN (missing) values by attribute for each DataFrame.
    - For attributes with 0% NaN values in both DataFrames, it compares their unique entries,
      reporting:
      - The count of common unique entries.
      - The count of unique entries present only in DataFrame 1 (ms).
      - The count of unique entries present only in DataFrame 2 (dwh).
    
    All values are converted to string type before comparison to prevent type mismatch issues.

    Args:
        df1 (pd.DataFrame): The first pandas DataFrame (e.g., ms).
        df2 (pd.DataFrame): The second pandas DataFrame (e.g., dwh).
    """

    print("--- Starting Gap Analysis between DataFrames ---")
    print(f"\nDataFrame 1 (ms) has {len(df1)} lines.")
    print(f"DataFrame 2 (dwh) has {len(df2)} lines.")

    common_attributes = [
        'id', 'sample_type', 'external_id', 'sample_code', 'fabrication_code',
        'batch_code_number', 'batch_code_supplier', 'sample_ec_code',
        'sample_smiles_code', 'created_at', 'updated_at'
    ]

    # --- NaN Value Analysis ---
    print("\n--- NaN Value Analysis by Attribute ---")
    nan_info = {} # Store NaN percentages for later use in value comparison
    for attr in common_attributes:
        nan_percent_df1 = (df1[attr].isnull().sum() / len(df1)) * 100 if len(df1) > 0 else 0
        nan_percent_df2 = (df2[attr].isnull().sum() / len(df2)) * 100 if len(df2) > 0 else 0
        nan_info[attr] = {'df1': nan_percent_df1, 'df2': nan_percent_df2} # Store percentages

        print(f"\nAttribute: '{attr}'")
        print(f"  - DataFrame 1 (ms): {nan_percent_df1:.2f}% NaN values")
        print(f"  - DataFrame 2 (dwh): {nan_percent_df2:.2f}% NaN values")

        if abs(nan_percent_df1 - nan_percent_df2) > 0.01:
            print(f"  * Note: There is a significant difference in NaN percentages for '{attr}'.")

    # --- Value Comparison for Complete Attributes ---
    print("\n--- Value Comparison for Attributes with 0% NaN ---")
    attributes_compared_for_values = False
    for attr in common_attributes:
        # Only compare values if the attribute has 0% NaN in *both* DataFrames
        if nan_info[attr]['df1'] == 0 and nan_info[attr]['df2'] == 0:
            attributes_compared_for_values = True
            print(f"\nAttribute: '{attr}' (0% NaN in both DataFrames)")

            # Convert column values to string type before converting to sets for consistent comparison
            set1 = set(df1[attr].astype(str).tolist())
            set2 = set(df2[attr].astype(str).tolist())

            common_unique_values = set1.intersection(set2)
            unique_only_in_df1 = set1.difference(set2)
            unique_only_in_df2 = set2.difference(set1)

            print(f"  - Number of common unique entries: {len(common_unique_values)}")
            print(f"  - Number of unique entries only in DataFrame 1 (pcp_sample_cas_ms): {len(unique_only_in_df1)}")
            print(f"  - Number of unique entries only in DataFrame 2 (pcp_sample_cas_dwh): {len(unique_only_in_df2)}")

            # Optionally, you could print the actual differing values for small sets:
            # if len(unique_only_in_df1) > 0 and len(unique_only_in_df1) < 6:
            #     print(f"    Values unique to DF1: {list(unique_only_in_df1)}")
            # if len(unique_only_in_df2) > 0 and len(unique_only_in_df2) < 6:
            #     print(f"    Values unique to DF2: {list(unique_only_in_df2)}")
        
    if not attributes_compared_for_values:
        print("  No attributes were found with 0% NaN values in both DataFrames to compare individual entries.")

    print("\n--- Gap Analysis Complete ---")

# SAMPLE PCP

## Source CAS

In [217]:
pcp_sample_cas_ms.sample(n=2)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
12218,39301,CAS,90045253,90045-25-3,NaN,NaN,NaN,NaN,NaN,2025-06-12T09:59:40.956Z,2025-06-12T09:59:40.956Z
12155,39364,CAS,90082125,90082-12-5,NaN,NaN,NaN,NaN,NaN,2025-06-12T10:17:47.576Z,2025-06-12T10:17:47.576Z


In [218]:
pcp_sample_cas_dwh.sample(n=2)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
6515,43723,CAS,None,17318-08-0,None,None,None,None,None,2025-06-16 20:23:19.286666+00:00,2025-06-16 20:23:19.286666+00:00
5980,35165,CAS,None,14431-43-7,None,None,None,None,None,2025-04-30 15:32:46.466666+00:00,2025-04-30 15:32:46.466666+00:00


In [219]:
gap_analysis_dataframes(pcp_sample_cas_ms, pcp_sample_cas_dwh)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 17391 lines.
DataFrame 2 (dwh) has 12560 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values
  * Note: There is a significant difference in NaN percentages for 'external_id'.

Attribute: 'sample_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'fabrication_code'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_number'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_supplier'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

In [220]:
# Entries in MS not in DWH 
pcp_sample_cas_ms[~(pcp_sample_cas_ms["external_id"].isin(pcp_sample_cas_dwh["external_id"]))].sample(n=5)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
27,51492,CAS,223106410,223106-41-0,NaN,NaN,NaN,NaN,NaN,2025-07-02T06:03:11.103Z,2025-07-02T06:03:11.103Z
804,50715,CAS,80181313,80181-31-3,NaN,NaN,NaN,NaN,NaN,2025-06-26T05:34:03.280Z,2025-06-26T05:34:03.280Z
4337,47182,CAS,97643,97-64-3,NaN,NaN,NaN,NaN,NaN,2025-06-20T15:28:45.683Z,2025-06-20T15:28:45.683Z
14435,37084,CAS,548083,548-08-3,NaN,NaN,NaN,NaN,NaN,2025-05-05T09:36:43.553Z,2025-05-05T09:36:43.553Z
12335,39184,CAS,164780830,164780-83-0,NaN,NaN,NaN,NaN,NaN,2025-06-12T09:21:26.190Z,2025-06-12T09:21:26.190Z


## Source CAS (source_code null inclus)

In [221]:
gap_analysis_dataframes(pcp_sample_cas_ms, pcp_sample_cas_dwh_sc_null_allowed)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 17391 lines.
DataFrame 2 (dwh) has 17398 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values
  * Note: There is a significant difference in NaN percentages for 'external_id'.

Attribute: 'sample_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'fabrication_code'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_number'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_supplier'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

In [222]:
# Entries in MS not in DWH 
try:
    pcp_sample_cas_ms[~(pcp_sample_cas_ms["external_id"].isin(pcp_sample_cas_dwh_sc_null_allowed["external_id"]))].sample(n=5)
except:
    print("MS contenu dans DWH")

In [223]:
# Entries in dwh not in ms
pcp_sample_cas_dwh_sc_null_allowed[~(pcp_sample_cas_dwh_sc_null_allowed["external_id"].isin(pcp_sample_cas_ms["external_id"]))].head(n=11)


,id,source_code,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
0,51524,None,CAS,None,1174327-61-7,None,None,None,None,None,2025-07-28 08:12:18.916666+00:00,2025-07-28 08:12:18.916666+00:00
1,37631,None,CAS,None,1174921-24-4,None,None,None,None,None,2025-05-20 08:09:17.653333+00:00,2025-05-20 08:09:17.653333+00:00
2,51517,None,CAS,None,1178566-95-4,None,None,None,None,None,2025-07-11 13:42:21.543333+00:00,2025-07-11 13:42:21.543333+00:00
3,34181,None,CAS,None,1314879-20-3,None,None,None,None,None,2025-04-28 15:08:17.940000+00:00,2025-04-28 15:08:17.940000+00:00
4,37632,None,CAS,None,1375969-44-0,None,None,None,None,None,2025-05-20 08:13:44.506666+00:00,2025-05-20 08:13:44.506666+00:00
5,34159,None,CAS,None,246159-33-1,None,None,None,None,None,2025-04-28 15:02:28.936666+00:00,2025-04-28 15:02:28.936666+00:00
6,34147,None,CAS,None,2639151-39-4,None,None,None,None,None,2025-04-28 14:59:28.593333+00:00,2025-04-28 14:59:28.593333+00:00
7,34150,None,CAS,None,3017930-93-4,None,None,None,None,None,2025-04-28 15:00:16.013333+00:00,2025-04-28 15:00:16.013333+00:00
8,37637,None,CAS,None,51999-21-4,None,None,None,None,None,2025-06-04 08:39:20.770000+00:00,2025-06-04 08:39:20.770000+00:00
9,51523,None,CAS,None,68958-56-5,None,None,None,None,None,2025-07-28 08:04:50.293333+00:00,2025-07-28 08:04:50.293333+00:00


## Source CAS (source_code null inclus, AA-CAS inclus)

In [224]:
gap_analysis_dataframes(pcp_sample_cas_ms, pcp_sample_cas_dwh_sc_null_allowed_aa_cas_allowed)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 17391 lines.
DataFrame 2 (dwh) has 17398 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values
  * Note: There is a significant difference in NaN percentages for 'external_id'.

Attribute: 'sample_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'fabrication_code'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_number'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_supplier'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

In [225]:
# Entries in MS not in DWH 
pcp_sample_cas_ms[~(pcp_sample_cas_ms["external_id"].isin(pcp_sample_cas_dwh_sc_null_allowed_aa_cas_allowed["external_id"]))].head(n=7)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
0,51519,CAS,84649821,84649-82-1,NaN,NaN,NaN,NaN,NaN,2025-07-17T12:24:20.303Z,2025-07-17T12:24:20.303Z
1,51518,CAS,5356843,5356-84-3,NaN,NaN,NaN,NaN,NaN,2025-07-17T12:20:26.920Z,2025-07-17T12:20:26.920Z
2,51517,CAS,1178566954,1178566-95-4,NaN,NaN,NaN,NaN,NaN,2025-07-11T13:42:21.543Z,2025-07-11T13:42:21.543Z
3,51516,CAS,31501118,31501-11-8,NaN,NaN,NaN,NaN,NaN,2025-07-11T09:36:25.713Z,2025-07-11T09:36:25.713Z
4,51515,CAS,68901224,68901-22-4,NaN,NaN,NaN,NaN,NaN,2025-07-10T08:58:54.903Z,2025-07-10T08:58:54.903Z
5,51514,CAS,156810,156-81-0,NaN,NaN,NaN,NaN,NaN,2025-07-08T15:49:56.733Z,2025-07-08T15:49:56.733Z
6,51513,CAS,35139674,35139-67-4,NaN,NaN,NaN,NaN,NaN,2025-07-08T15:32:16.310Z,2025-07-08T15:32:16.310Z


In [226]:
pcp_sample_cas_dwh_sc_null_allowed_aa_cas_allowed[~(pcp_sample_cas_dwh_sc_null_allowed_aa_cas_allowed["external_id"].isin(pcp_sample_cas_ms["external_id"]))].head(n=7)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
0,51396,CAS,None,10067-28-4,None,None,None,None,None,2025-06-30 08:55:02.383333+00:00,2025-06-30 08:55:02.383333+00:00
1,51525,CAS,None,109976-49-0,None,None,None,None,None,2025-07-30 08:21:28.520000+00:00,2025-07-30 08:21:28.520000+00:00
2,51524,CAS,None,1174327-61-7,None,None,None,None,None,2025-07-28 08:12:18.916666+00:00,2025-07-28 08:12:18.916666+00:00
3,37631,CAS,None,1174921-24-4,None,None,None,None,None,2025-05-20 08:09:17.653333+00:00,2025-05-20 08:09:17.653333+00:00
4,51517,CAS,None,1178566-95-4,None,None,None,None,None,2025-07-11 13:42:21.543333+00:00,2025-07-11 13:42:21.543333+00:00
5,51393,CAS,None,1189096-02-3,None,None,None,None,None,2025-06-27 14:43:56.190000+00:00,2025-06-27 14:43:56.190000+00:00
6,48544,CAS,None,1225570-30-8,None,None,None,None,None,2025-06-21 01:01:18.700000+00:00,2025-06-21 01:01:18.700000+00:00


## Source ECHA


In [227]:
pcp_sample_echa_ms.sample(n=2)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
3166,8928,CAS,a2106004-0b69-4c24-88e8-0496c8409bde,101-68-8,NaN,NaN,NaN,202-966-0,O=C=Nc1ccc(Cc2ccc(cc2)N=C=O)cc1,2025-06-16T06:44:53.213Z,2025-06-16T06:44:53.213Z
2426,9668,CAS,98e4131f-2e80-46cb-aee3-40a5d2dc3b0a,59487-23-9,NaN,NaN,NaN,261-785-5,COc1cc(OC)c(cc1Cl)NC(=O)c5cc2ccccc2c(N=Nc3cc(c...,2025-06-16T22:37:58.390Z,2025-06-16T22:37:58.390Z


In [228]:
pcp_sample_echa_dwh.sample(n=2)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
2580,9467,CAS,None,174393-75-0,None,None,None,None,,2025-06-16 20:20:01.743333+00:00,2025-06-16 20:20:01.743333+00:00
1084,8355,CAS,None,68187-76-8,None,None,None,269-123-7,Not applicable,2025-06-13 02:09:45.290000+00:00,2025-06-13 02:09:45.290000+00:00


In [229]:
gap_analysis_dataframes(pcp_sample_echa_ms, pcp_sample_echa_dwh)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 5152 lines.
DataFrame 2 (dwh) has 5153 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values
  * Note: There is a significant difference in NaN percentages for 'external_id'.

Attribute: 'sample_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'fabrication_code'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_number'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_supplier'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values



In [230]:
# Entries in MS not in DWH 
pcp_sample_echa_ms[~(pcp_sample_echa_ms["external_id"].isin(pcp_sample_echa_dwh["external_id"]))].head(65)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
0,12094,CAS,5d8a0da4-4511-4044-bcd0-7b503eed007f,35139-67-4,NaN,NaN,NaN,609-076-5,C1=C([N+](=C(N=C1Cl)N)[O-])N,2025-07-08T15:32:29.610Z,2025-07-08T15:32:29.610Z
1,12093,CAS,d26d5029-0024-4315-b548-343fd93d8303,93-92-5,NaN,NaN,NaN,202-288-5,CC(OC(=O)C)c1ccccc1,2025-07-02T06:10:24.770Z,2025-07-02T06:10:24.770Z
2,12092,CAS,1c50d7ff-85fb-4a7c-8bcb-e1dae86868a6,9000-90-2,NaN,NaN,NaN,232-565-6,NaN,2025-07-02T06:09:31.580Z,2025-07-02T06:09:31.580Z
3,12091,CAS,0a13b5cf-7888-4847-a4a3-9760d6f73aff,4706-78-9,NaN,NaN,NaN,225-190-4,S(OCCCCCCCCCCCC)(=O)([O-])=O.[K+],2025-07-02T06:07:43.236Z,2025-07-02T06:07:43.236Z
4,12090,CAS,0968b8aa-69f1-4e5b-87ae-0915f8fa98a5,4560-68-3,NaN,NaN,NaN,224-931-9,CCCCCCCCC(CCCCCC)CCC(=O)CC(O)(CC(=O)OCC(CCCCCC...,2025-07-02T06:06:52.623Z,2025-07-02T06:06:52.623Z
...,...,...,...,...,...,...,...,...,...,...,...
60,12034,CAS,e5729307-8208-44d3-9e55-e494646ff311,2814-77-9,NaN,NaN,NaN,220-562-2,Clc3cc(ccc3N=Nc1c2ccccc2ccc1O)[N+]([O-])=O,2025-06-27T09:40:10.490Z,2025-06-27T09:40:10.490Z
61,12033,CAS,45cc368d-b6a5-41dd-b88a-c09150ea77da,2580-56-5,NaN,NaN,NaN,219-943-6,NaN,2025-06-27T09:37:35.506Z,2025-06-27T09:37:35.506Z
62,12032,CAS,5718adbf-a1f7-4e6b-babe-17ed9fae3373,693-33-4,NaN,NaN,NaN,211-748-4,CCCCCCCCCCCCCCCC[N+](C)(C)CC(=O)[O-],2025-06-27T09:33:45.363Z,2025-06-27T09:33:45.363Z
63,12031,CAS,4613a548-91ed-44c5-b8f3-c5802a15b5ff,82576-75-8,NaN,NaN,NaN,NaN,NaN,2025-06-27T09:32:44.296Z,2025-06-27T09:32:44.296Z


In [231]:
# Entries in DWH not in MS
pcp_sample_echa_dwh[~(pcp_sample_echa_dwh["external_id"].isin(pcp_sample_echa_ms["external_id"]))].head(1)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
0,6957,CAS,None,1314-35-8,None,None,None,215-231-4,O=[W](=O)=O,2025-04-28 15:08:15.706666+00:00,2025-04-28 15:08:15.706666+00:00


## Source LIMS

In [232]:
pcp_sample_lims_ms.sample(n=2)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
7368,23432,Refcomm,798622,C39616538,DGA21D0002,DGA21D0002-2550112,2550112,NaN,NaN,2025-07-03T14:41:49.060Z,2025-07-03T17:03:15.682Z
2592,28987,Refcomm,670306,C61063764,NaN,R0070593A 007 X 007,NaN,NaN,NaN,2025-07-04T17:26:51.223Z,2025-07-04T17:28:27.212Z


In [233]:
pcp_sample_lims_dwh.sample(n=2)

,id,source_code,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
3188,22957,ANALYSE_AULNAY,Refcomm,768957,C22012415,DGA22D0351,DGA22D0351-3008634,3008634,None,None,2025-07-03 13:58:19.883333+00:00,2025-07-03 13:58:19.883333+00:00
2812,29687,ANALYSE_AULNAY,Refcomm,598575,B41061246,None,S-1506151IR,None,None,None,2025-07-04 19:16:52.083333+00:00,2025-07-04 19:16:55.704000+00:00


In [234]:
gap_analysis_dataframes(pcp_sample_lims_ms, pcp_sample_lims_dwh)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 8110 lines.
DataFrame 2 (dwh) has 8106 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'fabrication_code'
  - DataFrame 1 (ms): 35.34% NaN values
  - DataFrame 2 (dwh): 35.36% NaN values
  * Note: There is a significant difference in NaN percentages for 'fabrication_code'.

Attribute: 'batch_code_number'
  - DataFrame 1 (ms): 0.06% NaN values
  - DataFrame 2 (dwh): 0.06% NaN values

Attribute: 'batch_code_supplier'
  - DataFrame 1 (ms): 38.79% NaN values
  - DataFrame 2 (dwh): 38.80% NaN values

Attri

In [235]:
# Entries in DWH not in MS 
pcp_sample_lims_dwh[~(pcp_sample_lims_dwh["id"].isin(pcp_sample_lims_ms["id"]))].sample(n=5)

,id,source_code,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
2042,28700,ANALYSE_AULNAY,Refcomm,790773,B1181217,DGA2220226,DGA2220226-1521I036,1521I036,None,None,2025-07-04 14:52:36.863333+00:00,2025-07-04 14:52:37.830000+00:00
862,23075,ANALYSE_AULNAY,Refcomm,849774,51993750,B0000183838,B0000183838-194660,194660,None,None,2025-07-03 14:08:09.793333+00:00,2025-07-17 15:35:29.260000+00:00
5464,31499,ANALYSE_AULNAY,Refcomm,801073,C37994190,DGA2220322,DGA2220322-H052K9U153,H052K9U153,None,None,2025-07-07 12:33:40.800000+00:00,2025-07-17 12:24:13+00:00
3218,22745,ANALYSE_AULNAY,Refcomm,874630,C22017913,DGA2350113,DGA2350113-ESD0904215,ESD0904215,None,None,2025-07-03 13:44:42.396666+00:00,2025-07-04 14:53:29.796666+00:00
304,22741,ANALYSE_AULNAY,Refcomm,875027,46577119,B0000205997,B0000205997-777033,777033,None,None,2025-07-03 13:44:29.560000+00:00,2025-07-07 12:45:14.856666+00:00


In [236]:
pcp_sample_lims_dwh[~(pcp_sample_lims_dwh["id"].isin(pcp_sample_lims_ms["id"]))]["sample_type"].value_counts()

sample_type
Refcomm    11
Name: count, dtype: int64

In [237]:
# Entries in MS not in DWH 
pcp_sample_lims_ms[~(pcp_sample_lims_ms["external_id"].isin(pcp_sample_lims_dwh["external_id"]))].head()

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
0,31728,Refcomm,868156,54523314,DGC24N0271,DGC24N0271-0040273149,0040273149,NaN,NaN,2025-08-01T10:02:04.113Z,2025-08-01T10:05:50.404Z
1,31727,Refcomm,868155,71648662,B0000205769,B0000205769-2401202,2401202,NaN,NaN,2025-08-01T10:01:45.130Z,2025-08-01T10:05:54.567Z
2,31726,Refcomm,868153,53881194,B0000191808,B0000191808-07828230419,07828230419,NaN,NaN,2025-08-01T10:01:42.436Z,2025-08-01T10:05:56.419Z
3,31725,Refcomm,868154,50697682,B0000197307,B0000197307-3339,3339,NaN,NaN,2025-08-01T10:01:34.596Z,2025-08-01T10:05:52.591Z
4,31724,Refcomm,840517,73033935,B0000202372,B0000202372-25191,25191,NaN,NaN,2025-07-31T14:27:15.600Z,2025-07-31T14:27:15.600Z


## Source LIMS (sample_type CAS exclus)

In [238]:
gap_analysis_dataframes(pcp_sample_lims_ms, pcp_sample_lims_dwh_no_cas)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 8110 lines.
DataFrame 2 (dwh) has 8106 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'fabrication_code'
  - DataFrame 1 (ms): 35.34% NaN values
  - DataFrame 2 (dwh): 35.36% NaN values
  * Note: There is a significant difference in NaN percentages for 'fabrication_code'.

Attribute: 'batch_code_number'
  - DataFrame 1 (ms): 0.06% NaN values
  - DataFrame 2 (dwh): 0.06% NaN values

Attribute: 'batch_code_supplier'
  - DataFrame 1 (ms): 38.79% NaN values
  - DataFrame 2 (dwh): 38.80% NaN values

Attri

In [239]:
# Entries in DWH not in MS 
pcp_sample_lims_dwh_no_cas[~(pcp_sample_lims_dwh_no_cas["id"].isin(pcp_sample_lims_ms["id"]))].sample(n=2)

,id,source_code,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
304,22741,ANALYSE_AULNAY,Refcomm,875027,46577119,B0000205997,B0000205997-777033,777033,None,None,2025-07-03 13:44:29.560000+00:00,2025-07-07 12:45:14.856666+00:00
2041,28701,ANALYSE_AULNAY,Refcomm,790773,B1181217,DGA2220226,DGA2220226-1521I036,1521I036,None,None,2025-07-04 14:52:40.813333+00:00,2025-07-04 14:53:01.720000+00:00


In [240]:
# Entries in MS not in DWH 
pcp_sample_lims_ms[~(pcp_sample_lims_ms["external_id"].isin(pcp_sample_lims_dwh_no_cas["external_id"]))].sample(n=2)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
14,31714,Refcomm,876883,C40215087,DGA2430280,DGA2430280-26123EVFPF,26123EVFPF,NaN,NaN,2025-07-31T13:29:30.030Z,2025-07-31T13:29:30.030Z
12,31716,Refcomm,870703,C22265841,B0000203584,B0000203584-02C-150321Z,02C-150321Z,NaN,NaN,2025-07-31T13:41:35.570Z,2025-07-31T13:42:13.507Z


### Deep dive into one entry in DWH not in MS (id: 22716, external_id: 790773, sample_code: B1181217)

In [241]:
pcp_sample_lims_dwh_no_cas[pcp_sample_lims_dwh_no_cas["sample_code"]=="B1181217"]

,id,source_code,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
2039,28702,ANALYSE_AULNAY,Refcomm,790773,B1181217,DGA2220226,DGA2220226-1521I036,1521I036,None,None,2025-07-04 14:53:03.693333+00:00,2025-07-04 14:53:05.523000+00:00
2040,22716,ANALYSE_AULNAY,Refcomm,790773,B1181217,DGA2220226,DGA2220226-1521I036,1521I036,None,None,2025-07-03 13:42:30.086666+00:00,2025-07-04 14:52:33.656666+00:00
2041,28701,ANALYSE_AULNAY,Refcomm,790773,B1181217,DGA2220226,DGA2220226-1521I036,1521I036,None,None,2025-07-04 14:52:40.813333+00:00,2025-07-04 14:53:01.720000+00:00
2042,28700,ANALYSE_AULNAY,Refcomm,790773,B1181217,DGA2220226,DGA2220226-1521I036,1521I036,None,None,2025-07-04 14:52:36.863333+00:00,2025-07-04 14:52:37.830000+00:00


In [242]:
pcp_sample_lims_ms[pcp_sample_lims_ms["sample_code"]=="B1181217"]

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
2873,28702,Refcomm,790773,B1181217,DGA2220226,DGA2220226-1521I036,1521I036,NaN,NaN,2025-07-04T14:53:03.693Z,2025-07-04T14:53:05.523Z


Les 11 entrées dans le DWH pas dans le MS peuvent être expliqués par ça 

### Deep dive into one entry in MS not in DWH (id: 23851, external_id: 824825, sample_code: 71634281)

In [243]:
pcp_sample_lims_dwh_no_cas[pcp_sample_lims_dwh_no_cas["sample_code"]=="71634281"]

,id,source_code,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
1812,28680,ANALYSE_AULNAY,Refcomm,824730,71634281,B0000195906,B0000195906-K-64162,K-64162,None,None,2025-07-04 11:09:10.490000+00:00,2025-07-04 11:09:10.490000+00:00
1813,24169,ANALYSE_AULNAY,Refcomm,824779,71634281,B0000195906,B0000195906-K-64162,K-64162,None,None,2025-07-03 15:49:54.566666+00:00,2025-07-03 15:49:54.566666+00:00
1814,28697,ANALYSE_AULNAY,Refcomm,824761,71634281,B0000195906,B0000195906-K-64162,K-64162,None,None,2025-07-04 11:09:58.930000+00:00,2025-07-04 11:09:58.930000+00:00


In [244]:
pcp_sample_lims_ms[pcp_sample_lims_ms["sample_code"]=="71634281"]

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
2875,28697,Refcomm,824761,71634281,B0000195906,B0000195906-K-64162,K-64162,NaN,NaN,2025-07-04T11:09:58.930Z,2025-07-04T11:09:58.930Z
2892,28680,Refcomm,824730,71634281,B0000195906,B0000195906-K-64162,K-64162,NaN,NaN,2025-07-04T11:09:10.490Z,2025-07-04T11:09:10.490Z
6642,24169,Refcomm,824779,71634281,B0000195906,B0000195906-K-64162,K-64162,NaN,NaN,2025-07-03T15:49:54.566Z,2025-07-03T15:49:54.566Z


Il y a bien un id 23851 en DWH MSR mais son source_code est null ! Donc la clause source_code = "ANALYSE_AULNAY" le fait disparaître à tort

## Source LIMS (sample_type CAS exclus, source_code null inclus)

In [245]:
gap_analysis_dataframes(pcp_sample_lims_ms, pcp_sample_lims_dwh_no_cas_sc_null_allowed)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 8110 lines.
DataFrame 2 (dwh) has 9036 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 10.29% NaN values
  * Note: There is a significant difference in NaN percentages for 'external_id'.

Attribute: 'sample_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'fabrication_code'
  - DataFrame 1 (ms): 35.34% NaN values
  - DataFrame 2 (dwh): 39.49% NaN values
  * Note: There is a significant difference in NaN percentages for 'fabrication_code'.

Attribute: 'batch_code_number'
  - DataFrame 1 (ms): 0.06% NaN values
  - DataFrame 2 (dwh): 0.06% NaN values

Attribute: 'batch_code_supplier'
  - 

In [246]:
# Entries in MS not in DWH 
pcp_sample_lims_ms[~(pcp_sample_lims_ms["id"].isin(pcp_sample_lims_dwh_no_cas_sc_null_allowed["id"]))].head(13)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
0,31728,Refcomm,868156,54523314,DGC24N0271,DGC24N0271-0040273149,0040273149,NaN,NaN,2025-08-01T10:02:04.113Z,2025-08-01T10:05:50.404Z
1,31727,Refcomm,868155,71648662,B0000205769,B0000205769-2401202,2401202,NaN,NaN,2025-08-01T10:01:45.130Z,2025-08-01T10:05:54.567Z
2,31726,Refcomm,868153,53881194,B0000191808,B0000191808-07828230419,07828230419,NaN,NaN,2025-08-01T10:01:42.436Z,2025-08-01T10:05:56.419Z
3,31725,Refcomm,868154,50697682,B0000197307,B0000197307-3339,3339,NaN,NaN,2025-08-01T10:01:34.596Z,2025-08-01T10:05:52.591Z
4,31724,Refcomm,840517,73033935,B0000202372,B0000202372-25191,25191,NaN,NaN,2025-07-31T14:27:15.600Z,2025-07-31T14:27:15.600Z
5,31723,Refcomm,840518,73033941,B0000198570,B0000198570-501451,501451,NaN,NaN,2025-07-31T14:27:12.900Z,2025-07-31T14:27:12.900Z
6,31722,Refcomm,840516,73033909,B0000198572,B0000198572-51388,51388,NaN,NaN,2025-07-31T14:26:20.490Z,2025-07-31T14:26:20.490Z
7,31721,Refcomm,875972,222066,DGK2520187,DGK2520187-219H1A714,219H1A714,NaN,NaN,2025-07-31T13:53:05.830Z,2025-07-31T13:53:10.096Z
8,31720,Refcomm,870707,46492227,B0000204192,B0000204192-MSHP-14542,MSHP-14542,NaN,NaN,2025-07-31T13:41:54.913Z,2025-07-31T13:42:06.375Z
9,31719,Refcomm,870704,C22266094,B0000203583,B0000203583-05C-120523X,05C-120523X,NaN,NaN,2025-07-31T13:41:46.130Z,2025-07-31T13:42:11.656Z


In [247]:
# Entries in DWH not in MS 
pcp_sample_lims_dwh_no_cas_sc_null_allowed[~(pcp_sample_lims_dwh_no_cas_sc_null_allowed["id"].isin(pcp_sample_lims_ms["id"]))]\
    [["id","external_id","sample_code"]].head(11)

,id,external_id,sample_code
0,28189,None,105461
4,28181,None,117841
6,25952,None,140372
9,26352,None,145955
10,26035,None,145955
12,25990,None,145955
13,26040,None,145955
14,26017,None,145955
16,28419,None,147656
20,28177,None,150459


Some entries are missing external_id, are they exactly the ones without source_code ? 

In [248]:
entries_no_sc = pcp_sample_lims_dwh_no_cas_sc_null_allowed[(pcp_sample_lims_dwh_no_cas_sc_null_allowed["source_code"].isna())].copy()
entries_no_ext_id = pcp_sample_lims_dwh_no_cas_sc_null_allowed[(pcp_sample_lims_dwh_no_cas_sc_null_allowed["external_id"].isna())].copy()
inter = pd.merge(entries_no_sc, entries_no_ext_id, how="inner", left_on="id", right_on="id")

print(len(entries_no_sc))
print(len(entries_no_ext_id))
print(len(inter))


930
930
930


In [261]:
inter.head(10)

,id,source_code_x,sample_type_x,external_id_x,sample_code_x,fabrication_code_x,batch_code_number_x,batch_code_supplier_x,sample_ec_code_x,sample_smiles_code_x,...,sample_type_y,external_id_y,sample_code_y,fabrication_code_y,batch_code_number_y,batch_code_supplier_y,sample_ec_code_y,sample_smiles_code_y,created_at_y,updated_at_y
0,28189,None,Refcomm,None,105461,None,0003162482,None,None,None,...,Refcomm,None,105461,None,0003162482,None,None,None,2025-07-04 10:24:45.080000+00:00,2025-07-04 10:24:45.080000+00:00
1,28181,None,Refcomm,None,117841,None,350 306,None,None,None,...,Refcomm,None,117841,None,350 306,None,None,None,2025-07-04 10:24:26.986666+00:00,2025-07-04 10:24:26.986666+00:00
2,25952,None,Refcomm,None,140372,None,S080460012,None,None,None,...,Refcomm,None,140372,None,S080460012,None,None,None,2025-07-03 17:50:37.170000+00:00,2025-07-03 17:50:37.170000+00:00
3,26352,None,Refcomm,None,145955,SF-S12,SF-S12-,None,None,None,...,Refcomm,None,145955,SF-S12,SF-S12-,None,None,None,2025-07-04 07:02:59.256666+00:00,2025-07-04 07:02:59.256666+00:00
4,26035,None,Refcomm,None,145955,None,SF-S03,None,None,None,...,Refcomm,None,145955,None,SF-S03,None,None,None,2025-07-03 17:53:18.670000+00:00,2025-07-03 17:53:18.670000+00:00
5,25990,None,Refcomm,None,145955,None,091218,None,None,None,...,Refcomm,None,145955,None,091218,None,None,None,2025-07-03 17:51:54.793333+00:00,2025-07-03 17:51:54.793333+00:00
6,26040,None,Refcomm,None,145955,None,091218,None,None,None,...,Refcomm,None,145955,None,091218,None,None,None,2025-07-03 17:53:43.296666+00:00,2025-07-03 17:53:43.296666+00:00
7,26017,None,Refcomm,None,145955,None,SF-S03,None,None,None,...,Refcomm,None,145955,None,SF-S03,None,None,None,2025-07-03 17:52:45.793333+00:00,2025-07-03 17:52:45.793333+00:00
8,28419,None,Refcomm,None,147656,None,002012,None,None,None,...,Refcomm,None,147656,None,002012,None,None,None,2025-07-04 10:32:32.996666+00:00,2025-07-04 10:32:32.996666+00:00
9,28177,None,Refcomm,None,150459,None,211578 23,None,None,None,...,Refcomm,None,150459,None,211578 23,None,None,None,2025-07-04 10:24:20.033333+00:00,2025-07-04 10:24:20.033333+00:00


In [260]:
pcp_sample_lims_ms[pcp_sample_lims_ms['id']=="28181"]

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at


## Source COSMETOCHEM

In [251]:
gap_analysis_dataframes(pcp_sample_cosmetochem_ms, pcp_sample_cosmetochem_dwh)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 3368 lines.
DataFrame 2 (dwh) has 3372 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'sample_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values
  * Note: There is a significant difference in NaN percentages for 'external_id'.

Attribute: 'sample_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'fabrication_code'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_number'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values

Attribute: 'batch_code_supplier'
  - DataFrame 1 (ms): 100.00% NaN values
  - DataFrame 2 (dwh): 100.00% NaN values



In [252]:
# Entrées en DWH pas en MS
pcp_sample_cosmetochem_dwh[~(pcp_sample_cosmetochem_dwh["id"].isin(pcp_sample_cosmetochem_ms["id"]))].head(11)

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at
1,6928,CAS,None,109976-49-0,None,None,None,None,None,2025-07-30 08:23:56.866666+00:00,2025-07-30 08:23:56.866666+00:00
5,6927,CAS,None,183158-70-5,None,None,None,None,None,2025-07-25 11:22:52.740000+00:00,2025-07-25 11:22:52.740000+00:00
9,6926,CAS,None,32539-83-6,None,None,None,None,None,2025-07-23 14:25:47.430000+00:00,2025-07-23 14:25:47.430000+00:00
22,6925,CAS,None,87-78-5,None,None,None,None,None,2025-07-23 08:36:26.720000+00:00,2025-07-23 08:36:26.720000+00:00


In [253]:
pcp_sample_cosmetochem_ms[pcp_sample_cosmetochem_ms["id"]=="6926"]

,id,sample_type,external_id,sample_code,fabrication_code,batch_code_number,batch_code_supplier,sample_ec_code,sample_smiles_code,created_at,updated_at


# TEST PCP

## GA function test

In [254]:
def gap_analysis_dataframes_test(df1: pd.DataFrame, df2: pd.DataFrame):
    """
    Performs a comprehensive gap analysis between two pandas DataFrames.

    This function provides useful information to assess the data gaps, including:
    - Number of lines in both DataFrames.
    - Percentage of NaN (missing) values by attribute for each DataFrame.
    - For attributes with 0% NaN values in both DataFrames, it compares their unique entries,
      reporting:
      - The count of common unique entries.
      - The count of unique entries present only in DataFrame 1 (ms).
      - The count of unique entries present only in DataFrame 2 (dwh).
    
    All values are converted to string type before comparison to prevent type mismatch issues.

    Args:
        df1 (pd.DataFrame): The first pandas DataFrame (e.g., ms).
        df2 (pd.DataFrame): The second pandas DataFrame (e.g., dwh).
    """

    print("--- Starting Gap Analysis between DataFrames ---")
    print(f"\nDataFrame 1 (ms) has {len(df1)} lines.")
    print(f"DataFrame 2 (dwh) has {len(df2)} lines.")

    common_attributes = ['external_id', 'tested_sample_id',
       'measure_objective_type_code', 'test_sub_type', 'test_name',
       'test_variation', 'test_version', 'test_validation_date', 'created_at',
       'updated_at']

    # --- NaN Value Analysis ---
    print("\n--- NaN Value Analysis by Attribute ---")
    nan_info = {} # Store NaN percentages for later use in value comparison
    for attr in common_attributes:
        nan_percent_df1 = (df1[attr].isnull().sum() / len(df1)) * 100 if len(df1) > 0 else 0
        nan_percent_df2 = (df2[attr].isnull().sum() / len(df2)) * 100 if len(df2) > 0 else 0
        nan_info[attr] = {'df1': nan_percent_df1, 'df2': nan_percent_df2} # Store percentages

        print(f"\nAttribute: '{attr}'")
        print(f"  - DataFrame 1 (ms): {nan_percent_df1:.2f}% NaN values")
        print(f"  - DataFrame 2 (dwh): {nan_percent_df2:.2f}% NaN values")

        if abs(nan_percent_df1 - nan_percent_df2) > 0.01:
            print(f"  * Note: There is a significant difference in NaN percentages for '{attr}'.")

    # --- Value Comparison for Complete Attributes ---
    print("\n--- Value Comparison for Attributes with 0% NaN ---")
    attributes_compared_for_values = False
    for attr in common_attributes:
        # Only compare values if the attribute has 0% NaN in *both* DataFrames
        if nan_info[attr]['df1'] == 0 and nan_info[attr]['df2'] == 0:
            attributes_compared_for_values = True
            print(f"\nAttribute: '{attr}' (0% NaN in both DataFrames)")

            # Convert column values to string type before converting to sets for consistent comparison
            set1 = set(df1[attr].astype(str).tolist())
            set2 = set(df2[attr].astype(str).tolist())

            common_unique_values = set1.intersection(set2)
            unique_only_in_df1 = set1.difference(set2)
            unique_only_in_df2 = set2.difference(set1)

            print(f"  - Number of common unique entries: {len(common_unique_values)}")
            print(f"  - Number of unique entries only in DataFrame 1 (pcp_sample_cas_ms): {len(unique_only_in_df1)}")
            print(f"  - Number of unique entries only in DataFrame 2 (pcp_sample_cas_dwh): {len(unique_only_in_df2)}")

            # Optionally, you could print the actual differing values for small sets:
            # if len(unique_only_in_df1) > 0 and len(unique_only_in_df1) < 6:
            #     print(f"    Values unique to DF1: {list(unique_only_in_df1)}")
            # if len(unique_only_in_df2) > 0 and len(unique_only_in_df2) < 6:
            #     print(f"    Values unique to DF2: {list(unique_only_in_df2)}")
        
    if not attributes_compared_for_values:
        print("  No attributes were found with 0% NaN values in both DataFrames to compare individual entries.")

    print("\n--- Gap Analysis Complete ---")

## Processing

In [255]:
pcp_test_lims_ms.head(2)

,id,measure_objective_type_id,measure_objective_type_code,external_id,tested_sample_id,source_id,source_code,method_id,method_code,test_type,...,device_id,input_structure_smiles,clean_structure_smiles,version,reviewer_role,created_at,updated_at,deleted_at,tested_operating_parameters,individual_measurements
0,60154,1,DENSITY_01,1750255,31726,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:05:56.476Z,2025-08-01T10:05:56.476Z,NaN,NaN,NaN
1,60153,1,DENSITY_01,1750257,31727,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:05:54.583Z,2025-08-01T10:05:54.583Z,NaN,NaN,NaN


In [256]:
pcp_test_lims_dwh.head(2)

,id,external_id,tested_sample_id,measure_objective_type_code,test_sub_type,test_name,test_variation,test_version,test_validation_date,created_at,updated_at
0,43215,1719507,22678,TURBIDITY_GEL,MEAS_PHYS_CHEM,MEAS_TURBIDITY,None,1,2025-07-02 11:19:54+00:00,2025-07-03 13:37:28.670000+00:00,2025-07-03 13:37:28.670000+00:00
1,43216,1743283,22679,DENSITY_01,MEAS_PHYS_CHEM,MEAS_DENSITY,POWDER,3,2025-06-30 15:46:53+00:00,2025-07-03 13:37:33.390000+00:00,2025-07-03 13:37:33.390000+00:00


In [257]:
gap_analysis_dataframes_test(pcp_test_lims_ms, pcp_test_lims_dwh)

--- Starting Gap Analysis between DataFrames ---

DataFrame 1 (ms) has 15651 lines.
DataFrame 2 (dwh) has 15660 lines.

--- NaN Value Analysis by Attribute ---

Attribute: 'external_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'tested_sample_id'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'measure_objective_type_code'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'test_sub_type'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'test_name'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN values

Attribute: 'test_variation'
  - DataFrame 1 (ms): 31.58% NaN values
  - DataFrame 2 (dwh): 31.48% NaN values
  * Note: There is a significant difference in NaN percentages for 'test_variation'.

Attribute: 'test_version'
  - DataFrame 1 (ms): 0.00% NaN values
  - DataFrame 2 (dwh): 0.00% NaN 

Pourquoi le DWH dépasse sur le MS ? Des données inactives mal filtrées ? Quelques samples concentrent plusieurs entrées en écart

In [258]:
# Inspect entries in DWH not in MS 
pcp_test_lims_dwh[~(pcp_test_lims_dwh["id"].isin(pcp_test_lims_ms["id"]))].head()

,id,external_id,tested_sample_id,measure_objective_type_code,test_sub_type,test_name,test_variation,test_version,test_validation_date,created_at,updated_at
48,43263,1704205,22702,TURBIDITY_GEL,MEAS_PHYS_CHEM,MEAS_TURBIDITY,None,1,2025-06-23 14:15:32+00:00,2025-07-03 13:39:57.700000+00:00,2025-07-04 09:19:09.136666+00:00
107,43322,1719506,22715,VISCOSITY_VS_CONCENTRATION,MEAS_PHYS_CHEM,MEAS_VISCOSITY,VISCOSITY_VS_CONCENT,4,2025-06-22 14:29:26+00:00,2025-07-03 13:42:24.866666+00:00,2025-07-04 08:17:58.170000+00:00
108,43323,1732235,22716,VISCOSITY_VS_CONCENTRATION,MEAS_PHYS_CHEM,MEAS_VISCOSITY,VISCOSITY_VS_CONCENT,4,2025-06-22 13:47:22+00:00,2025-07-03 13:42:30.140000+00:00,2025-07-04 14:52:33.660000+00:00
109,43324,1611267,22717,TURBIDITY_GEL,MEAS_PHYS_CHEM,MEAS_TURBIDITY,None,1,2025-06-22 14:22:16+00:00,2025-07-03 13:42:41.933333+00:00,2025-07-04 14:53:18.750000+00:00
111,43326,1611268,22717,VISCOSITY_VS_CONCENTRATION,MEAS_PHYS_CHEM,MEAS_VISCOSITY,VISCOSITY_VS_CONCENT,3,2025-06-22 14:22:16+00:00,2025-07-03 13:42:57.436666+00:00,2025-07-04 14:53:18.753333+00:00


In [259]:
# Inspect entries in MS not in DWH (if consecutive IDs dated today its probably just due to DWH lag)
pcp_test_lims_ms[~(pcp_test_lims_ms["id"].isin(pcp_test_lims_dwh["id"]))].head(10)

,id,measure_objective_type_id,measure_objective_type_code,external_id,tested_sample_id,source_id,source_code,method_id,method_code,test_type,...,device_id,input_structure_smiles,clean_structure_smiles,version,reviewer_role,created_at,updated_at,deleted_at,tested_operating_parameters,individual_measurements
0,60154,1,DENSITY_01,1750255,31726,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:05:56.476Z,2025-08-01T10:05:56.476Z,NaN,NaN,NaN
1,60153,1,DENSITY_01,1750257,31727,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:05:54.583Z,2025-08-01T10:05:54.583Z,NaN,NaN,NaN
2,60152,1,DENSITY_01,1750256,31725,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:05:52.610Z,2025-08-01T10:05:52.610Z,NaN,NaN,NaN
3,60151,1,DENSITY_01,1750258,31728,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:05:50.490Z,2025-08-01T10:05:50.490Z,NaN,NaN,NaN
4,60150,1,DENSITY_01,1724126,31726,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:02:22.633Z,2025-08-01T10:02:22.633Z,NaN,NaN,NaN
5,60149,1,DENSITY_01,1724141,31728,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:02:20.386Z,2025-08-01T10:02:20.386Z,NaN,NaN,NaN
6,60147,2,WET_POINT,1724144,31728,6,ANALYSE_AULNAY,86.0,METHOD_NO_INFO,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:02:09.616Z,2025-08-01T10:02:09.616Z,NaN,NaN,NaN
7,60146,1,DENSITY_01,1724136,31727,6,ANALYSE_AULNAY,79.0,METHOD_CID_025_00_URL,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:02:07.883Z,2025-08-01T10:02:07.883Z,NaN,NaN,NaN
8,60144,2,WET_POINT,1724129,31726,6,ANALYSE_AULNAY,86.0,METHOD_NO_INFO,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:02:02.170Z,2025-08-01T10:02:02.170Z,NaN,NaN,NaN
9,60142,2,WET_POINT,1724139,31727,6,ANALYSE_AULNAY,86.0,METHOD_NO_INFO,Mesure,...,NaN,NaN,NaN,NaN,AU_RES_DEM,2025-08-01T10:01:45.153Z,2025-08-01T10:01:45.153Z,NaN,NaN,NaN
